In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

import numpy as np

import matplotlib.pyplot as plt

In [ ]:
print('Tensorflow: {}\nNumpy: {}'.format(tf.__version__, np.version.version))

**Load the IMDB dataset**

In [ ]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

**About the data**

In [ ]:
print('Training data: {}, labels: {}'.format(len(train_data), len(train_labels)))

In [ ]:
print(train_data[0])

In [ ]:
len(train_data[0]), len(train_data[1])

**Converting integers back to words**

In [ ]:
word_index = imdb.get_word_index()

In [ ]:
word_index = {k:(v+3) for k,v in word_index.items()}
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<UNUSED>'] = 3

In [ ]:
reverse_word_index = dict([(value, key) for  (key, value) in word_index.items()])

In [ ]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [ ]:
decode_review(train_data[0])

**Prepare the data**

Since the movie reviews must be the same length, we will use the pad_sequences function to standardize the lengths.



In [ ]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index['<PAD>'], padding='post', maxlen=256)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index['<PAD>'], padding='post', maxlen=256)

In [ ]:
len(train_data[0]), len(train_data[1])

In [ ]:
print(train_data[0])

**Build the model**

In [ ]:
VOCAB_SIZE = 10000

In [ ]:
model = keras.Sequential()

In [ ]:
model.add(keras.layers.Embedding(VOCAB_SIZE, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

In [ ]:
model.summary()

**Compile the model**

In [ ]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

**Create validation set**

In [ ]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

In [ ]:
y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

**Training the mode**

In [ ]:
history = model.fit(partial_x_train, partial_y_train, epochs=40, batch_size=512, validation_data=(x_val, y_val), verbose=1)

**Evaluation**

In [ ]:
#@title
model.evaluate(test_data, test_labels)

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
accuracy = history_dict['accuracy']
val_accuracy = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

In [ ]:
epochs = range(1, len(accuracy) + 1)

In [ ]:
plt.xlabel('Epochs')

plt.plot(epochs, accuracy, 'g--', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'g', label='Validation accuracy')

plt.plot(epochs, loss, 'r--', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')

plt.legend()
plt.show()